In [1]:
import numpy as np
from keras import utils
from keras.optimizers import Adam
from tensorflow.keras.models import load_model
from keras.datasets import cifar10
from keras.datasets import cifar100

In [2]:
dataset1 = cifar10
(x_train1, y_train1), (x_test1, y_test1) = dataset1.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test1[0]        
#translate the data to image
x_train1 = x_train1.astype("float32")
x_test1 = x_test1.astype("float32")


#normalization
x_train1 /= 255
x_test1 /= 255

alextnet_10 = load_model('Model/alexnet_cifar10.h5')#load lenet5 model trained with mnist dataset 
alextnet_10.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 

In [14]:
beta1 = 0.03998
sample(x_test1,y_test1,alextnet_10,beta1)

Beta is reduced by 0.0001, the new beta is 0.03988
Beta is reduced by 0.0001, the new beta is 0.039779999999999996
Beta is reduced by 0.0001, the new beta is 0.03967999999999999
Beta is reduced by 0.0001, the new beta is 0.03957999999999999
Beta is reduced by 0.0001, the new beta is 0.03947999999999999
Beta is reduced by 0.0001, the new beta is 0.039379999999999984
Beta is reduced by 0.0001, the new beta is 0.03927999999999998
Beta is reduced by 0.0001, the new beta is 0.03917999999999998
Beta is reduced by 0.0001, the new beta is 0.039079999999999976
Beta is reduced by 0.0001, the new beta is 0.03897999999999997
Beta is reduced by 0.0001, the new beta is 0.03887999999999997
Beta is reduced by 0.0001, the new beta is 0.03877999999999997
Beta is reduced by 0.0001, the new beta is 0.038679999999999964
Beta is reduced by 0.0001, the new beta is 0.03857999999999996
Beta is reduced by 0.0001, the new beta is 0.03847999999999996
Beta is reduced by 0.0001, the new beta is 0.038379999999999956

0.02027999999999982

In [15]:
dataset2 = cifar100
(x_train2, y_train2), (x_test2, y_test2) = dataset2.load_data() #original size train(xxx, 28, 28),test(xxx)

img = x_test2[0]        
#translate the data to image
x_train2 = x_train2.astype("float32")
x_test2 = x_test2.astype("float32")


#normalization
x_train2 /= 255
x_test2 /= 255

alextnet_100 = load_model('Model/alexnet_cifar100.h5')#load lenet5 model trained with mnist dataset 
alextnet_100.compile(optimizer=Adam(),loss='sparse_categorical_crossentropy',metrics=['accuracy']) 


In [21]:
beta2 = 0.02050
sample(x_test2,y_test2,alextnet_100,beta2)

Beta is reduced by 0.0001, the new beta is 0.0204
Beta is reduced by 0.0001, the new beta is 0.020300000000000002
Beta is reduced by 0.0001, the new beta is 0.020200000000000003
Beta is reduced by 0.0001, the new beta is 0.020100000000000003
Beta is reduced by 0.0001, the new beta is 0.020000000000000004
Beta is reduced by 0.0001, the new beta is 0.019900000000000004
Beta is reduced by 0.0001, the new beta is 0.019800000000000005
Beta is reduced by 0.0001, the new beta is 0.019700000000000006
Beta is reduced by 0.0001, the new beta is 0.019600000000000006
Beta is reduced by 0.0001, the new beta is 0.019500000000000007
Beta is reduced by 0.0001, the new beta is 0.019400000000000008
Beta is reduced by 0.0001, the new beta is 0.019300000000000008
Beta is reduced by 0.0001, the new beta is 0.01920000000000001
Beta is reduced by 0.0001, the new beta is 0.01910000000000001
Beta is reduced by 0.0001, the new beta is 0.01900000000000001
Beta is reduced by 0.0001, the new beta is 0.018900000000

0.010300000000000063

In [5]:
def sample(x_test,y_test,model, beta):
    mean = np.zeros(3)  
    cov = np.eye(3)
    
    for i in range(x_test.shape[0]):
        count = 0
        while True:
            p = np.random.multivariate_normal(mean, cov,(32,32)) #sample formula
            neighbour = p[0] + x_test[i]*255
            neighbour = neighbour.reshape(-1,32,32,3)
            neighbour /= 255
            distance=np.linalg.norm(p[0]) 
            distance /= 255
            #print(distance)
            if (distance < beta or distance == beta):
                output = model.predict(neighbour,batch_size=128)
                proba = softmax(output)
                pred = np.argmax(proba)
                label = y_test[i]
                if(pred == label):
                    count+=1
                else:  #if there are some iamges inside beta cannot have the same label, beta will be decreased
                    beta = beta-0.0001
                    print("Beta is reduced by 0.0001, the new beta is",beta)  
                    i-=1
                    break
                    
            if(count<600):  
                continue
                
            if(count==600):  
                #local robustness formula satisfied
                break
                
    print("global robustness formula satisfied")     
    print("max safe distance:",beta)
    return beta

def softmax(x):
    e_x = np.exp(x-np.max(x))
    return e_x / e_x.sum()